In [ ]:
pip install rake_nltk

In [4]:
import pandas as pd
from datetime import datetime
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from rake_nltk import Rake
import operator
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
movies=pd.read_csv('IMDb movies.csv')

C:\Users\sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
imdb=movies.dropna()

In [7]:
imdb.drop(columns=['date_published','usa_gross_income','writer','production_company',
                     'language','country','votes','budget','worlwide_gross_income','reviews_from_users','reviews_from_critics','metascore'],inplace=True)

C:\Users\sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6616 entries, 506 to 85740
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_title_id   6616 non-null   object 
 1   title           6616 non-null   object 
 2   original_title  6616 non-null   object 
 3   year            6616 non-null   object 
 4   genre           6616 non-null   object 
 5   duration        6616 non-null   int64  
 6   director        6616 non-null   object 
 7   actors          6616 non-null   object 
 8   description     6616 non-null   object 
 9   avg_vote        6616 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 568.6+ KB


In [9]:
imdb['year']=pd.to_datetime(imdb['year'], format='%Y')

C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/2867114007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb['year']=pd.to_datetime(imdb['year'], format='%Y')


In [10]:
imdb.year.unique()

array(['1927-01-01T00:00:00.000000000', '1931-01-01T00:00:00.000000000',
       '1936-01-01T00:00:00.000000000', '1937-01-01T00:00:00.000000000',
       '1939-01-01T00:00:00.000000000', '1940-01-01T00:00:00.000000000',
       '1941-01-01T00:00:00.000000000', '1942-01-01T00:00:00.000000000',
       '2001-01-01T00:00:00.000000000', '1946-01-01T00:00:00.000000000',
       '1948-01-01T00:00:00.000000000', '1950-01-01T00:00:00.000000000',
       '1951-01-01T00:00:00.000000000', '1952-01-01T00:00:00.000000000',
       '1953-01-01T00:00:00.000000000', '1954-01-01T00:00:00.000000000',
       '1956-01-01T00:00:00.000000000', '1955-01-01T00:00:00.000000000',
       '1957-01-01T00:00:00.000000000', '1958-01-01T00:00:00.000000000',
       '1959-01-01T00:00:00.000000000', '1963-01-01T00:00:00.000000000',
       '1960-01-01T00:00:00.000000000', '1961-01-01T00:00:00.000000000',
       '1962-01-01T00:00:00.000000000', '1964-01-01T00:00:00.000000000',
       '1966-01-01T00:00:00.000000000', '1965-01-01

In [11]:
imdb.loc[(imdb.loc[:, 'year'] =='TV Movie 2019')]='2019'

C:\Users\sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [12]:
imdb['year']=pd.to_datetime(imdb['year'], format='%Y')

C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/2867114007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb['year']=pd.to_datetime(imdb['year'], format='%Y')


In [13]:
imdb.head()

,imdb_title_id,title,original_title,year,genre,duration,director,actors,description,avg_vote
506,tt0017136,Metropolis,Metropolis,1927-01-01,"Drama, Sci-Fi",153,Fritz Lang,"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",In a futuristic city sharply divided between t...,8.3
1048,tt0021749,Luci della città,City Lights,1931-01-01,"Comedy, Drama, Romance",87,Charles Chaplin,"Virginia Cherrill, Florence Lee, Harry Myers, ...","With the aid of a wealthy erratic tippler, a d...",8.5
2454,tt0027977,Tempi moderni,Modern Times,1936-01-01,"Comedy, Drama, Family",87,Charles Chaplin,"Charles Chaplin, Paulette Goddard, Henry Bergm...",The Tramp struggles to live in modern industri...,8.5
2795,tt0029453,Il bandito della Casbah,Pépé le Moko,1937-01-01,"Crime, Drama, Romance",94,Julien Duvivier,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fe...",A wanted gangster is both king and prisoner of...,7.7
2827,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937-01-01,"Animation, Family, Fantasy",83,"William Cottrell, David Hand","Roy Atwell, Stuart Buchanan, Adriana Caselotti...",Exiled into the dangerous forest by her wicked...,7.6


In [14]:
Key_words =[]
r = Rake()
for index, row in imdb.iterrows():
    r.extract_keywords_from_text(row['description'])
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words'] = list(key_words_dict_scores.keys())
    Key_words.append(row['Key_words'])
imdb['Key_words']=Key_words

C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/1534484078.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb['Key_words']=Key_words


In [15]:
imdb['Genre'] = imdb['genre'].map(lambda x: x.split(','))
imdb['Actors'] = imdb['actors'].map(lambda x: x.split(',')[:3])
imdb['Director'] = imdb['director'].map(lambda x: x.split(','))

genre=[]
actors=[]
director=[]

for index, row in imdb.iterrows():
    row['Genre'] = [x.lower().replace(' ','') for x in row['Genre']]
    genre.append(row['Genre'])
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    actors.append(row['Actors'])
    row['Director'] = [x.lower().replace(' ','') for x in row['Director']]
    director.append(row['Director'])
    
imdb['Genre']=genre
imdb['Actors']=actors
imdb['Director']=director

C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/2813023411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb['Genre'] = imdb['genre'].map(lambda x: x.split(','))
C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/2813023411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb['Actors'] = imdb['actors'].map(lambda x: x.split(',')[:3])
C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/2813023411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [75]:
imdb

,imdb_title_id,title,original_title,year,genre,duration,director,actors,description,avg_vote,Key_words,Genre,Actors,Director,Bag_of_words
506,tt0017136,Metropolis,Metropolis,1927-01-01,"Drama, Sci-Fi",153,Fritz Lang,"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",In a futuristic city sharply divided between t...,8.3,"[futuristic, city, sharply, divided, working, ...","[drama, sci-fi]","[alfredabel, gustavfröhlich, rudolfklein-rogge]",[fritzlang],drama sci-fi fritzlang alfredabel gustavfröhli...
1048,tt0021749,Luci della città,City Lights,1931-01-01,"Comedy, Drama, Romance",87,Charles Chaplin,"Virginia Cherrill, Florence Lee, Harry Myers, ...","With the aid of a wealthy erratic tippler, a d...",8.5,"[aid, wealthy, erratic, tippler, dewy, eyed, t...","[comedy, drama, romance]","[virginiacherrill, florencelee, harrymyers]",[charleschaplin],comedy drama romance charleschaplin virginiach...
2454,tt0027977,Tempi moderni,Modern Times,1936-01-01,"Comedy, Drama, Family",87,Charles Chaplin,"Charles Chaplin, Paulette Goddard, Henry Bergm...",The Tramp struggles to live in modern industri...,8.5,"[tramp, struggles, live, modern, industrial, s...","[comedy, drama, family]","[charleschaplin, paulettegoddard, henrybergman]",[charleschaplin],comedy drama family charleschaplin charleschap...
2795,tt0029453,Il bandito della Casbah,Pépé le Moko,1937-01-01,"Crime, Drama, Romance",94,Julien Duvivier,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fe...",A wanted gangster is both king and prisoner of...,7.7,"[wanted, gangster, king, prisoner, casbah, pro...","[crime, drama, romance]","[jeangabin, gabrielgabrio, saturninfabre]",[julienduvivier],crime drama romance julienduvivier jeangabin g...
2827,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937-01-01,"Animation, Family, Fantasy",83,"William Cottrell, David Hand","Roy Atwell, Stuart Buchanan, Adriana Caselotti...",Exiled into the dangerous forest by her wicked...,7.6,"[exiled, dangerous, forest, wicked, stepmother...","[animation, family, fantasy]","[royatwell, stuartbuchanan, adrianacaselotti]","[williamcottrell, davidhand]",animation family fantasy williamcottrell david...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85266,tt9208444,Impractical Jokers: The Movie,Impractical Jokers: The Movie,2020-01-01,Comedy,92,Chris Henchy,"Paula Abdul, Joe Gatto, James Murray, Brian Qu...",The story of a humiliating high school mishap ...,5.9,"[story, humiliating, high, school, mishap, 199...",[comedy],"[paulaabdul, joegatto, jamesmurray]",[chrishenchy],comedy chrishenchy paulaabdul joegatto jamesmu...
85274,tt9214832,Emma.,Emma.,2020-01-01,"Comedy, Drama",124,Autumn de Wilde,"Anya Taylor-Joy, Angus Imrie, Letty Thomas, Ge...","In 1800s England, a well meaning but selfish y...",6.7,"[1800s, england, well, meaning, selfish, young...","[comedy, drama]","[anyataylor-joy, angusimrie, lettythomas]",[autumndewilde],comedy drama autumndewilde anyataylor-joy angu...
85397,tt9354944,Jexi,Jexi,2019-01-01,"Comedy, Romance",84,"Jon Lucas, Scott Moore","Adam Devine, Alexandra Shipp, Rose Byrne, Ron ...",A comedy about what can happen when you love y...,6.1,"[comedy, happen, love, phone, anything, else, ...","[comedy, romance]","[adamdevine, alexandrashipp, rosebyrne]","[jonlucas, scottmoore]",comedy romance jonlucas scottmoore adamdevine ...
85460,tt9426210,Weathering with You,Tenki no ko,2019-01-01,"Animation, Drama, Family",112,Makoto Shinkai,"Kotaro Daigo, Nana Mori, Shun Oguri, Sei Hirai...",A high-school boy who has run away to Tokyo be...,7.6,"[high, school, boy, run, away, tokyo, befriend...","[animation, drama, family]","[kotarodaigo, nanamori, shunoguri]",[makotoshinkai],animation drama family makotoshinkai kotarodai...


In [76]:
Bag_of_words=[]
columns = ['Genre', 'Director', 'Actors', 'Key_words']
for index, row in imdb.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    Bag_of_words.append(row['Bag_of_words'])
    
imdb['Bag_of_words']=Bag_of_words

C:\Users\sofia\AppData\Local\Temp/ipykernel_15140/2210319627.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb['Bag_of_words']=Bag_of_words


In [77]:
df = imdb[['title','original_title','year','genre','director','actors','avg_vote','Bag_of_words']]
df=df.reset_index()
df.drop(columns=['index'],inplace=True)

In [78]:
cm=CountVectorizer().fit_transform(df['Bag_of_words'])

In [79]:
cosine_sim =cosine_similarity(cm)
print(cosine_sim)

[[1.         0.08164966 0.04588315 ... 0.05       0.04588315 0.08      ]
 [0.08164966 1.         0.28097574 ... 0.20412415 0.14048787 0.08164966]
 [0.04588315 0.28097574 1.         ... 0.11470787 0.10526316 0.04588315]
 ...
 [0.05       0.20412415 0.11470787 ... 1.         0.         0.1       ]
 [0.04588315 0.14048787 0.10526316 ... 0.         1.         0.04588315]
 [0.08       0.08164966 0.04588315 ... 0.1        0.04588315 1.        ]]


In [80]:
cosine_sim.shape

(6616, 6616)

In [81]:
indices = pd.Series(df['original_title'])

In [84]:
def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:15].index)

    for i in top_10_indices:
        recommended_movies.append(list(df['original_title'])[i])
        
    return recommended_movies

In [92]:
recommend('Harry Potter and the Prisoner of Azkaban')

['Harry Potter and the Half-Blood Prince',
 "Harry Potter and the Sorcerer's Stone",
 'Harry Potter and the Deathly Hallows: Part 2',
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter and the Deathly Hallows: Part 1',
 'Fantastic Beasts and Where to Find Them',
 'A Little Princess',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Chamber of Secrets',
 'The Wiz',
 'The Nutcracker and the Four Realms',
 'Ferdinand',
 'The Wizard of Oz',
 'Alice in Wonderland']

In [29]:
indices_1 = pd.Series(df['genre'])

In [88]:
def recommend_genero(genero, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices_1[indices_1 == genero].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:15].index)

    for i in top_10_indices:
        recommended_movies.append(list(df['original_title'])[i])
        
    return recommended_movies

In [89]:
 recommend_genero('Drama')

['Last Orders',
 'Extreme Prejudice',
 'Mystic River',
 'Cape of Good Hope',
 'Another Year',
 'Autumn in New York',
 'Emma.',
 'La mala educación',
 'Danny Collins',
 'Ballast',
 'To Save a Life',
 'Book Club',
 'The Death & Life of John F. Donovan',
 'Cleopatra']

In [42]:
indices_2 = pd.Series(df['director'])

In [90]:
def recommend_director(Director, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices_2[indices_2 == Director].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:15].index)

    for i in top_10_indices:
        recommended_movies.append(list(df['original_title'])[i])
        
    return recommended_movies

In [93]:
recommend_director('Steven Spielberg')

['Down Terrace',
 'Funny Games',
 'Canvas',
 "The Devil's Rejects",
 'Road to Perdition',
 'Breaking In',
 'In the Bedroom',
 'Double Jeopardy',
 'Black and Blue',
 'Donkey Punch',
 'Birth',
 'The Longest Yard',
 'Heaven',
 'Casa de Areia']